# KKCC_주보_제작_노트북_v1.0

## GitHub 저장소
본 노트북 관련 QR 및 리소스는 아래 저장소에서 관리됩니다.

👉 [KKCC GitHub 저장소](https://github.com/jewook-park/KKCC/)

---

## 구조 설명

이 노트북은 KKCC 주보 및 성가 문서를 동시에 생성합니다.

### ① 미사 자료 수집
굿뉴스 미사 페이지에서 h3 기준으로:
- 제1독서
- 제2독서
- 복음
- 보편 지향 기도
- 영성체송

을 순서대로 수집합니다.

### ② QR 처리
GitHub raw PNG를 직접 다운로드하여 사용합니다.

### ③ 성가 처리
- 추천 성가 자동 추출
- 단성 우선, 없으면 혼성
- 1장만 사용
- 4페이지 구성
- 머리말: 입당/봉헌/성체/파견 성가 N번
- 꼬리말: YYYY-MM-DD 미사 · KKCC

---



# 주보/성가 자동 생성 — Colab 노트북 v10.2

## 이번 수정(요청 반영)
1) **주보**: 공지사항을 **영성체송 다음(마지막)** 섹션으로 이동  
   - 즉, 온라인 안내 → 독서/복음/보편/영성체송 → **공지사항(맨 마지막)**

2) **성가**: 머리말이 전부 '파견'으로 동일하게 나오는 문제 수정  
   - 원인: 새 섹션이 이전 섹션의 **header가 링크(linked_to_previous)** 되어 있어 마지막 값으로 덮이는 현상  
   - 해결: **각 섹션마다 header 링크를 해제**(`sec.header.is_linked_to_previous = False`)하고 해당 페이지 머리말을 개별 설정  
   - 4페이지(4섹션): **입당 → 봉헌 → 성체 → 파견** 순서 보장  
   - 꼬리말은 공통: `{MASS_DATE} 미사 · KKCC` (footer는 이전과 링크 유지)

QR raw:
- 홈페이지: https://raw.githubusercontent.com/jewook-park/KKCC/main/QR_KKCC_homepage.png
- 페이스북: https://raw.githubusercontent.com/jewook-park/KKCC/main/QR_KKCC_facebook.png
- 오늘의 말씀: https://raw.githubusercontent.com/jewook-park/KKCC/main/%EC%95%84%EC%98%A4%EC%8A%A4%ED%8C%85%EC%8B%A0%EB%B6%80%EB%8B%98_%EC%98%A4%EB%8A%98%EC%9D%98%EB%A7%90%EC%94%80QRcode.png


In [ ]:
!pip -q install python-docx requests beautifulsoup4 lxml pillow


In [ ]:

import re, io, requests
from bs4 import BeautifulSoup, Tag
from PIL import Image

from docx import Document
from docx.shared import Pt, Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.section import WD_SECTION
from docx.oxml import OxmlElement
from docx.oxml.ns import qn

from google.colab import files

MASS_DATE = "2026-02-22"
BULLETIN_OUT = f"주보_{MASS_DATE}.docx"
SONG_OUT = f"성가_{MASS_DATE}.docx"

MISSA_URL = "https://maria.catholic.or.kr/mi_pr/missa/missa.asp?menu=missa&missaid=16383&gomonth=2026-02-22&missatype=DA"

QR_HOME = "https://raw.githubusercontent.com/jewook-park/KKCC/main/QR_KKCC_homepage.png"
QR_FB = "https://raw.githubusercontent.com/jewook-park/KKCC/main/QR_KKCC_facebook.png"
QR_TODAY = "https://raw.githubusercontent.com/jewook-park/KKCC/main/%EC%95%84%EC%98%A4%EC%8A%A4%ED%8C%85%EC%8B%A0%EB%B6%80%EB%8B%98_%EC%98%A4%EB%8A%98%EC%9D%98%EB%A7%90%EC%94%80QRcode.png"

# (요청) 보편지향기도가 영성체송보다 앞
READING_SECTIONS = ["제1독서","제2독서","복음","보편 지향 기도","영성체송"]


## 공지사항 입력

In [ ]:
NOTICE_TEXT = """
1. (여기에 공지사항을 입력하세요)
2. (여기에 공지사항을 입력하세요)
"""


## QR 다운로드 (GitHub raw 사용)

In [ ]:
def download_png(url: str, out: str) -> str:
    r = requests.get(url, headers={"User-Agent":"Mozilla/5.0"}, timeout=30)
    r.raise_for_status()
    im = Image.open(io.BytesIO(r.content)).convert("RGB")
    im.save(out, format="PNG")
    return out

QR_HOME_PATH = download_png(QR_HOME, "QR_home.png")
QR_FB_PATH = download_png(QR_FB, "QR_fb.png")
QR_TODAY_PATH = download_png(QR_TODAY, "QR_today.png")


## 미사 본문 추출 (v10.1 유지)

In [ ]:
def fetch_html(url: str) -> str:
    r = requests.get(url, headers={"User-Agent":"Mozilla/5.0"}, timeout=30)
    r.raise_for_status()
    return r.text

html = fetch_html(MISSA_URL)
soup = BeautifulSoup(html, "lxml")

def norm(s: str) -> str:
    return re.sub(r"[ \t\u00a0]+", " ", s).strip()

def is_section_h3(h3: Tag):
    t = norm(h3.get_text())
    if t in READING_SECTIONS:
        return t
    if t.replace(" ","") == "보편지향기도":
        return "보편 지향 기도"
    return None

def collect_text(h3: Tag):
    lines=[]
    for sib in h3.next_siblings:
        if isinstance(sib, Tag) and sib.name == "h3":
            break
        if isinstance(sib, Tag):
            txt = sib.get_text("\n", strip=True)
            if txt:
                lines.append(txt)
    return "\n".join(lines)

sections={}
for h3 in soup.find_all("h3"):
    name = is_section_h3(h3)
    if name:
        sections[name]=collect_text(h3)

print("본문 섹션:", list(sections.keys()))


## 주보 DOC 생성 (공지사항을 맨 마지막으로 이동 + 온라인 안내 표/QR 정렬 유지)

In [ ]:
def add_center(doc, text):
    p = doc.add_paragraph()
    p.alignment = WD_ALIGN_PARAGRAPH.CENTER
    r = p.add_run(text)
    r.bold = True
    r.font.size = Pt(18)

def add_h1(doc, text):
    p = doc.add_paragraph()
    r = p.add_run(text)
    r.bold = True
    r.font.size = Pt(14)

def add_body(doc, text):
    doc.add_paragraph(text)

def set_table_borders_none(table):
    tbl = table._tbl
    tblPr = tbl.tblPr
    if tblPr is None:
        tblPr = OxmlElement('w:tblPr')
        tbl.insert(0, tblPr)

    tblBorders = OxmlElement('w:tblBorders')
    for edge in ("top","left","bottom","right","insideH","insideV"):
        el = OxmlElement(f"w:{edge}")
        el.set(qn("w:val"), "nil")
        tblBorders.append(el)
    tblPr.append(tblBorders)

def build_bulletin():
    doc = Document()
    add_center(doc, f"{MASS_DATE} 미사 주보")
    doc.add_paragraph("")

    
    # 1) 미사 본문
    for key in READING_SECTIONS:
        add_h1(doc, key)
        add_body(doc, sections.get(key, "(자료 없음)"))
        doc.add_paragraph("")

    # 2) 공지사항
    add_h1(doc, "공지사항")
    for line in NOTICE_TEXT.strip().splitlines():
        add_body(doc, line)
    doc.add_paragraph("")

    # 3) 온라인 안내 (공지사항 뒤)
    add_h1(doc, "온라인 안내")
    table = doc.add_table(rows=2, cols=3)
    table.autofit = True
    set_table_borders_none(table)

    labels = ["홈페이지", "페이스북", "오늘의 말씀"]
    urls = [
        "https://sites.google.com/site/catholicknox/home",
        "https://www.facebook.com/kkcatholic",
        "https://sites.google.com/site/catholicknox/home",
    ]
    qrs = [QR_HOME_PATH, QR_FB_PATH, QR_TODAY_PATH]

    for i in range(3):
        c = table.cell(0, i)
        p = c.paragraphs[0]
        p.text = ""
        r1 = p.add_run(labels[i] + "
")
        r1.bold = True
        r1.font.size = Pt(11)
        r2 = p.add_run(urls[i])
        r2.font.size = Pt(9)

        cq = table.cell(1, i)
        pq = cq.paragraphs[0]
        pq.alignment = WD_ALIGN_PARAGRAPH.CENTER
        run = pq.add_run()
        run.add_picture(qrs[i], width=Inches(1.15))

    doc.save(BULLETIN_OUT)
    return BULLETIN_OUT


b = build_bulletin()
print("주보 생성:", b)
files.download(b)


## 성가: 추천 성가 번호 추출

In [ ]:
ROLES = ["입당","예물준비","영성체","파견"]

def parse_hymns(soup):
    out = {r: [] for r in ROLES}
    text = soup.get_text(" ", strip=True)
    for role in ROLES:
        m = re.search(rf"{role}\s*성가\s*(.*?)(?=(?:{'|'.join(ROLES)})\s*성가|$)", text)
        if m:
            nums = re.findall(r"(\d{1,4})\s*번", m.group(1))
            uniq=[]
            for n in nums:
                if n not in uniq:
                    uniq.append(n)
            out[role]=uniq
    return out

rec = parse_hymns(soup)
selected = {r:(rec[r][0] if rec[r] else None) for r in ROLES}
print("선택 성가:", selected)


## 성가: 악보 다운로드 로직 (v8.3 복원)

In [ ]:
def sungga_view_url(num: str) -> str:
    return f"https://maria.catholic.or.kr/sungga/search/sungga_view.asp?ctxtIndex={num}"

def abs_url(u: str) -> str:
    if u.startswith("//"):
        return "https:" + u
    if u.startswith("/"):
        return "https://maria.catholic.or.kr" + u
    return u

def collect_img_urls(sp: BeautifulSoup, label: str | None = None) -> list[str]:
    urls=[]
    roots=[]
    if label:
        node = sp.find(string=lambda s: isinstance(s, str) and label in s)
        if node and node.parent:
            roots.append(node.parent)
            if node.parent.parent:
                roots.append(node.parent.parent)
    if not roots:
        roots=[sp]

    for root in roots:
        for tag in root.find_all(["img","a"]):
            u = tag.get("src") or tag.get("href") or ""
            if "/sungga/Files/img/" in u:
                urls.append(abs_url(u))

    if label:
        for tag in sp.find_all(["img","a"]):
            u = tag.get("src") or tag.get("href") or ""
            if "/sungga/Files/img/" in u:
                urls.append(abs_url(u))

    out=[]; seen=set()
    for u in urls:
        if u not in seen:
            out.append(u); seen.add(u)
    return out

def download_and_convert_to_png(session: requests.Session, img_url: str, referer: str, out_png: str) -> str | None:
    r = session.get(
        img_url,
        headers={"User-Agent":"Mozilla/5.0", "Referer": referer},
        allow_redirects=True,
        timeout=30,
    )
    if r.status_code != 200:
        return None
    ctype = (r.headers.get("Content-Type","") or "").lower()
    if not ctype.startswith("image/"):
        return None
    try:
        im = Image.open(io.BytesIO(r.content))
        im = im.convert("RGB")
        im.save(out_png, format="PNG")
        return out_png
    except Exception:
        return None

def get_first_score_png(song_num: str, session: requests.Session) -> tuple[str, str | None]:
    view_url = sungga_view_url(song_num)
    rv = session.get(view_url, headers={"User-Agent":"Mozilla/5.0"}, timeout=30)
    rv.raise_for_status()
    sp = BeautifulSoup(rv.text, "lxml")

    for label in ["단성","혼성"]:
        urls = collect_img_urls(sp, label=label)
        for i,u in enumerate(urls, start=1):
            out_png = f"{song_num}_{label}_{i}.png"
            saved = download_and_convert_to_png(session, u, referer=view_url, out_png=out_png)
            if saved:
                return (label, saved)
    return ("none", None)

song_sess = requests.Session()


## 성가 DOC 생성: 4섹션(4페이지) + 페이지별 머리말 + 공통 꼬리말

In [ ]:
def ensure_first_paragraph(container):
    if not container.paragraphs:
        return container.add_paragraph()
    return container.paragraphs[0]

def set_section_header(section, text: str):
    # 핵심: 섹션마다 header를 이전 섹션과 unlink 해야 페이지별로 다르게 유지됩니다.
    section.header.is_linked_to_previous = False
    hdr = section.header
    p = ensure_first_paragraph(hdr)
    p.text = text
    p.alignment = WD_ALIGN_PARAGRAPH.LEFT
    # 기존 run이 남아있을 수 있어 새로 만든 텍스트에 대해 run 폰트를 안전하게 지정
    for run in p.runs:
        run.font.size = Pt(11)
        run.bold = True

def set_section_footer_common(section, text: str):
    # footer는 공통 유지: 새 섹션은 footer를 previous와 링크한 상태로 둡니다.
    section.footer.is_linked_to_previous = True
    ftr = section.footer
    p = ensure_first_paragraph(ftr)
    p.text = text
    p.alignment = WD_ALIGN_PARAGRAPH.CENTER
    for run in p.runs:
        run.font.size = Pt(9)

def build_song_doc():
    doc = Document()

    # 첫 섹션 footer는 직접 세팅(이후 섹션들은 링크)
    doc.sections[0].footer.is_linked_to_previous = False
    ftr0 = doc.sections[0].footer
    p0 = ensure_first_paragraph(ftr0)
    p0.text = f"{MASS_DATE} 미사 · KKCC"
    p0.alignment = WD_ALIGN_PARAGRAPH.CENTER
    for run in p0.runs:
        run.font.size = Pt(9)

    role_map = [("입당","입당"),("봉헌","예물준비"),("성체","영성체"),("파견","파견")]

    for idx, (disp, key) in enumerate(role_map):
        if idx == 0:
            sec = doc.sections[0]
        else:
            sec = doc.add_section(WD_SECTION.NEW_PAGE)
            # footer 공통 링크 유지
            sec.footer.is_linked_to_previous = True

        num = selected.get(key)
        header_text = f"{disp} 성가 {num}번" if num else f"{disp} 성가"
        set_section_header(sec, header_text)

        if num:
            label, png = get_first_score_png(str(num), song_sess)
            if png:
                doc.add_picture(png, width=Inches(6.8))
                doc.paragraphs[-1].alignment = WD_ALIGN_PARAGRAPH.CENTER
            else:
                doc.add_paragraph(f"(악보 다운로드 실패: {sungga_view_url(num)})")
        else:
            doc.add_paragraph("(추천 성가 없음)")

    doc.save(SONG_OUT)
    return SONG_OUT

s = build_song_doc()
print("성가 생성:", s)
files.download(s)
